# Quantum Spectral Graph Convolutional Neural Networks


### Project Description:

Over recent years, a large influx of interest has been observed in classical machine learning regarding the research into and usage of Graph Neural Networks (GNN). Part of the reason for this interest is due to their innate ability to model vast physical phenomena through the medium of pair-wise interactions between the elements of the systems. Similarly , interest in Quantum Machine Learning models is also increasing, as such architectures can leverage the computational efficiency of quantum computers and offer problem tailored solutions by [handcrafting antsatze guided by physical interactions](https://arxiv.org/abs/2006.11287). Consequently, we believe that combining these separate ideas will offer mutual benefits and improve model performance and advanced research in both fields. Seeing how GNNs are used to solve combinatorial tasks [Combinatorial optimisation and reasoning with graph neural networks by Cappart et al](https://arxiv.org/abs/2102.09544) included in workshops such as [“Deep Learning and Combinatorial Optimisation”](https://www.ipam.ucla.edu/programs/workshops/deep-learning-and-combinatorial-optimization/) help at IPAM UCLA., we would argue that it is the right time to start thinking more about Quantum Graph Neural Networks (QGNN).

We propose to implement Quantum Spectral Graph Convolutional Neural Networks (QSGCNN) as described in [Verdon et al.](https://arxiv.org/abs/1909.12264). We are planning to use the [Pennylane documentation on Quantum Graph Recurrent Neural Networks (QGRNNs)](https://pennylane.ai/qml/demos/qgrnn.html) as a guideline, and we will replace the RNN layer with a spectral convolutional layer. In particular, we want to perform unsupervised graph clustering as described in [Verdon et al.](https://arxiv.org/abs/1909.12264). We specifically want to compare the performance and inference speed between classical GNN models and their quantum counterparts on simple datasets, such as the one in [Verdon et al.](https://arxiv.org/abs/1909.12264) or k-core distilled popular GNN benchmark datasets (e.g. Cora, or Citeseer). This would primarily include the most popular and basic models based on the SGCNNs and as a stretch goal also on GraphSAGE. The results would be then compared with standard graph partitioning algorithms.

### Achievements:

- set up a target hamiltonian 
- set up a qgcnn layer 
- set up a training function to find the low energy states of the hamiltonian

### Future work: 

- use the predicted weights to find the eigenvectors and eigenvalues of the Laplacian matrix and then generate cluster predictions from there
- change the ansatz for better training: QAOA ansatz or ArbitraryUnitary gates
- train our model on a bigger input graph
- set up a comparison to classical models


### Implementation: 

Let's first load the relevant packages:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pennylane as qml
from matplotlib import pyplot as plt
from pennylane import numpy as np
import scipy
import networkx as nx
import copy
from qgscnn import CouplingGate


The input to our model is the graph that is to be clustered. In the current example, the input is given as a weighted adjacency matrix:

In [3]:
weighted_graph_adj_matrix = np.array([
    [0., 1., 2., 1.],
    [1., 0., 3., 2.],
    [2., 3., 0., 1.],
    [1., 2., 1., 0.]
])

Spectral clustering uses the Graph Laplacian for our weighted graph. Given the weighted adjacency matrix, we calculate the Laplacian matrix: 

In [4]:
def compute_L_matrix(adj_matrix):
    vertex_sums = np.sum(adj_matrix, axis=0)
    out = np.zeros(adj_matrix.shape)
    for j in range(len(adj_matrix)):
        for k in range(len(adj_matrix)):
            if j != k:
                out[j][k] = -1. * adj_matrix[j][k]
            else:
                out[j][k] = vertex_sums[j] - adj_matrix[j][k]
                
    return out

Our target Hamiltonian is $H_C + H_A$, the coupling Hamiltonian plus the anharmonic hamiltonian. Our current implementation uses single qubit precision. The resulting matrix has dimension $(2^N, 2^N)$ with N being the number of nodes in the input graph. To be able to use Pennylane to calculate the expected value of the target Hamiltonian, the Hamiltonian is then converted into a Pennylane Hamiltonian object.

In [5]:
def target_hamiltonian(L_matrix):
    out = np.zeros((2**len(L_matrix),2**len(L_matrix)))
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            if i == j or L_matrix[i][j]==0.:
                pass
            else:
                m = 1
                for k in range(len(L_matrix)):
                    if k == i or k == j:
                        outer_prod = np.array([
                            [0.,0.],
                            [0., 1.]
                        ])

                        m = np.kron(m, outer_prod)
                    else:
                        m = np.kron(m, np.identity(2))
                               
                out += L_matrix[i][j] * m
            
    return out + np.identity(2**len(L_matrix))

def target_to_qml_hamiltonian(target):
    """
    Helper function to turn a matrix representing the target Hamiltonian into a Pennylane Hamiltonian object
    """
    pauli_coeffs, obs_list = qml.utils.decompose_hamiltonian(target)
    return qml.Hamiltonian(pauli_coeffs, obs_list)

Each layer of our Quantum Graph Spectral CNN first entangles the qubits and then rotates them according to our trainable parameters. 

In [6]:
def qgscnn_layer(weights, wires, L_matrix):
    for i in range(len(L_matrix)):
        for j in range(len(L_matrix)):
            if i != j:
                if L_matrix[i][j] != 0.:
                    qml.CNOT(wires=[i,j])

    # add X rotations for the kinetic hamiltonian
    for i in range(len(L_matrix)):
        qml.RX(weights[0][i], wires=i)
    
    
def qgscnn(weights, wires, L_matrix, num_layers, steps):
    for i in range(num_layers):
        qgscnn_layer(weights[i], wires, L_matrix)

We can now define the main function that trains the model. First, the Laplacian is computed, and the hamiltonian is set up. Steps and number of layers are hyperparameters of the model, weights are the parameters. Our ansatz is simply the QGSCNN. As our cost function, we use Pennylane's ExpvalCost, which automatically computes the expectation value of the input Hamiltonian given ansatz. The model is then trained using the optimier, and the original and trained weights are returned. 

In [9]:
steps = 20
num_layers = 5  

def quantum_clustering(dev, adj_matrix):
    # compute the L matrix
    L_matrix = compute_L_matrix(adj_matrix)
    
    # helps the cost fn if we declare how many vertices graph has
    num_vertices = len(adj_matrix)
    
    # compute the target Hamiltonian
    target = target_hamiltonian(L_matrix)
    target_H = target_to_qml_hamiltonian(target)
    
    # model set up
    optimiser = qml.AdamOptimizer(stepsize=0.5)
    
    # Defines the new QNode
    qnode = qml.QNode(qgscnn, dev)

    def ansatz(params, **kwargs):
        return qgscnn(params, wires=range(len(adj_matrix)), L_matrix=L_matrix, num_layers=num_layers, steps=steps)

    cost_fn = qml.ExpvalCost(ansatz, target_H, dev)
    
    # to make life easier, kinetic weights are weights[layer][1], and coupling weights are weights[layer][0]
    weights = np.random.randint(-20, 20, size=(num_layers, 1, adj_matrix.shape[1]))/50
    init = copy.copy(weights)

    # Executes the optimization method
    iterations = 0

    for i in range(0, steps):
        print(f"Iteration {i}")
        weights = optimiser.step(cost_fn, weights)
        print('Cost:', cost_fn(weights))
    
    return weights, init

This cell runs the training function.

In [8]:
dev = qml.device("default.qubit", wires=len(weighted_graph_adj_matrix))
L_matrix = compute_L_matrix(weighted_graph_adj_matrix)

weights, init = quantum_clustering(dev, weighted_graph_adj_matrix)

Iteration 0
Cost: -3.4701612262412027
Iteration 1
Cost: -4.5560974868725825
Iteration 2
Cost: -2.7561811560897596
Iteration 3
Cost: -1.4997994765249036
Iteration 4
Cost: -4.766243131574999
Iteration 5
Cost: -8.28916493876731
Iteration 6
Cost: -8.797616009775599
Iteration 7
Cost: -6.142868603268044
Iteration 8
Cost: -4.173928648954365
Iteration 9
Cost: -3.6625723374557744
Iteration 10
Cost: -4.33442992854978
Iteration 11
Cost: -6.819477062231077
Iteration 12
Cost: -10.409950802063308
Iteration 13
Cost: -12.009858492404327
Iteration 14
Cost: -12.855649318549327
Iteration 15
Cost: -14.320022717649469
Iteration 16
Cost: -16.087617501276327
Iteration 17
Cost: -15.407001794349123
Iteration 18
Cost: -14.603353771309793
Iteration 19
Cost: -14.64112101527788
Iteration 20
Cost: -14.792099434556567
Iteration 21
Cost: -14.757446907142395
Iteration 22
Cost: -15.300629107599004
Iteration 23
Cost: -17.03983009003577
Iteration 24
Cost: -18.311099052436834
Iteration 25
Cost: -17.717333641421458
Iterati

KeyboardInterrupt: 

To predict cluster values, we first return the quantum state of the QGSCNN. This is then used to calculate the eigenvalues of the Laplacian.

In [ ]:
@qml.qnode(dev)
def predict(weights):
    wires = range(len(weighted_graph_adj_matrix))
    qgscnn(weights, wires, L_matrix, num_layers, steps)
    return qml.state()

Now we calculate the eigenvalues.

In [ ]:
def get_eigenvalues(weights):
    state = np.array(predict(weights))
    target = target_hamiltonian(L_matrix)
    target_H = target_to_qml_hamiltonian(target)
    eigval_ham = np.matmul(target, state)
    return eigval_ham / state
    
print(get_eigenvalues(weights))
print(get_eigenvalues(init))

And compare the results to classical numerical functions:

In [ ]:
np.linalg.eig(target_hamiltonian(L_matrix))
np.linalg.eig(L_matrix)
np.linalg.eig(weighted_graph_adj_matrix)
